In [1]:
import pickle
import random
import numpy as np

In [2]:
# X = {file_name: (frames, joints (xyz) * 25)}
with open('X.pkl', 'rb') as f:
    X_raw = pickle.load(f)

In [3]:
train_x = {}
test_x = {}

# [{actor1: [frame], actor2: [frame]}]
train = []
test = []
same_samples_per_actor = 10000
diff_samples_per_actor = 20000

for file in X_raw:
    actor = int(file[9:12])
    action = int(file[17:20])
    
    is_train = action <= 60
    if is_train:
        if actor not in train_x:
            train_x[actor] = []
        if len(X_raw[file]) == 0:
            continue
        train_x[actor].append(X_raw[file])
    else:
        if actor not in test_x:
            test_x[actor] = []
        if len(X_raw[file]) == 0:
            continue
        test_x[actor].append(X_raw[file])

for actor in train_x:
    for i in range(same_samples_per_actor):
        # Randomly select 2 frames from the same actor
        video1 = random.choice(train_x[actor])
        video2 = random.choice(train_x[actor])
        frame1 = random.choice(video1)
        frame2 = random.choice(video2)
        train.append({actor: np.array([frame1, frame2]), 'is_same': True})

    for i in range(diff_samples_per_actor):
        # Randomly select 2 frames from different actors
        while True: # Make sure the 2 frames are not from the same actor
            random_actor = random.choice(list(train_x.keys()))
            if random_actor != actor:
                video1 = random.choice(train_x[actor])
                video2 = random.choice(train_x[random_actor])
                frame1 = random.choice(video1)
                frame2 = random.choice(video2)
                train.append({actor: frame1, random_actor: frame2, 'is_same': False})
                break

for actor in test_x:
    for i in range(same_samples_per_actor):
        # Randomly select 2 frames from the same actor
        video1 = random.choice(test_x[actor])
        video2 = random.choice(test_x[actor])
        frame1 = random.choice(video1)
        frame2 = random.choice(video2)
        test.append({actor: np.array([frame1, frame2]), 'is_same': True})

    for i in range(diff_samples_per_actor):
        # Randomly select 2 frames from different actors
        while True: # Make sure the 2 frames are not from the same actor
            random_actor = random.choice(list(test_x.keys()))
            if random_actor != actor:
                video1 = random.choice(test_x[actor])
                video2 = random.choice(test_x[random_actor])
                frame1 = random.choice(video1)
                frame2 = random.choice(video2)
                test.append({actor: frame1, random_actor: frame2, 'is_same': False})
                break

train = np.array(train)
test = np.array(test)


In [4]:
# Train
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

X_train = []
y_train = []

for sample in train:
    if sample['is_same']:
        for actor in sample:
            if actor != 'is_same':
                X_train.append(np.concatenate([sample[actor][0], sample[actor][1]]))
        y_train.append(1)
    else:
        temp = []
        for actor in sample:
            if actor != 'is_same':
                temp.append(sample[actor])
        X_train.append(np.concatenate(temp))
        y_train.append(0)

X_train = np.array(X_train)
y_train = np.array(y_train)

# I'll do a grid search later
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
clf.fit(X_train, y_train)

# Test
X_test = []
y_test = []

for sample in test:
    if sample['is_same']:
        for actor in sample:
            if actor != 'is_same':
                X_test.append(np.concatenate([sample[actor][0], sample[actor][1]]))
        y_test.append(1)
    else:
        temp = []
        for actor in sample:
            if actor != 'is_same':
                temp.append(sample[actor])
        X_test.append(np.concatenate(temp))
        y_test.append(0)

X_test = np.array(X_test)
y_test = np.array(y_test)

y_pred = clf.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1 Score: ', f1_score(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))

Accuracy:  0.6709251207729469
Precision:  0.6007912369365868
Recall:  0.03807536231884058
F1 Score:  0.07161227101559295
Confusion Matrix:
 [[1362543   17457]
 [ 663728   26272]]
